In [2]:
import pandas as pd, numpy as np
# import cudf,cupy
import matplotlib.pyplot as plt, gc, os

In [3]:
# CONSTANTS
MISSINGNESS_THRESHOLD = 0.99
TRAIN= '/content/drive/MyDrive/train_fe_v3_loaded_rectified.parquet' #remove MyDrive if running in Kaggle/local
TEST = '/content/drive/MyDrive/test_fe_v3_loaded_rectified (1).parquet'  #remove MyDrive if running in Kaggle/local
correlation_threshold=0.9
point_biserial_threshold=0.01
stat_sig=0.05

In [4]:
# Load
def load_data(datalink = ''):
    data = pd.read_parquet(datalink)
    #data['customer_ID'] = data['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
    data['customer_ID'] = data['customer_ID'].apply(lambda x: int(x[-16:], 16))
    print("Shape on loading: ", data.shape)
    return data

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
trainBase = load_data(TRAIN)

Shape on loading:  (458913, 2710)


# **Missing Values Removal**

In [6]:
def missing_columns(data):
    missing_percentages = data.isnull().mean()
    missing_columns = missing_percentages[missing_percentages > MISSINGNESS_THRESHOLD].index
    return missing_columns

In [7]:
def drop_columns_with_many_missing_values_in_derived_dataset(data,base_columns_to_drop):
  columns_to_drop=[]
  for i in range(0,len(base_columns_to_drop)):
    columns_to_drop.append(base_columns_to_drop[i]+'_')
  columns_to_drop = [col for col in data.columns if any(word in col for word in columns_to_drop)]
  data_filtered = data.drop(columns=columns_to_drop, errors='ignore')
  return data_filtered

In [8]:

missing_cols_original=['D_73', 'D_88', 'D_110', 'B_39', 'B_42']


In [9]:
trainBase=drop_columns_with_many_missing_values_in_derived_dataset(trainBase,missing_cols_original)

In [10]:
trainBase.shape

(458913, 2630)

In [ ]:
trainBase.columns

Index(['customer_ID', 'P_2_first', 'P_2_mean', 'P_2_std', 'P_2_min', 'P_2_max',
       'P_2_last', 'D_39_first', 'D_39_mean', 'D_39_std',
       ...
       'D_143_first_last_lag_div_round2', 'D_143_mean_last_lag_sub_round2',
       'D_143_mean_last_lag_div_round2', 'D_144_first_last_lag_sub_round2',
       'D_144_first_last_lag_div_round2', 'D_144_mean_last_lag_sub_round2',
       'D_144_mean_last_lag_div_round2', 'D_145_first_last_lag_div_round2',
       'D_145_mean_last_lag_sub_round2', 'D_145_mean_last_lag_div_round2'],
      dtype='object', length=2630)

# New Aggregates, DateTime and Inter-Category Variables

In [11]:
def more_agg_inter_intra_variables(df):
    # df=df.to_pandas()
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df=df.fillna(-127)
    num_features = [col for col in df.columns if col not in ['customer_ID', 'S_2', 'target']]
    cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
    num_features = [col for col in num_features if col not in cat_features and col.endswith('_last')]
    for col in num_features:
        df[f'{col}_log_last'] = np.log1p(df[f'{col}'])
    P_columns = [col for col in df.columns if col.startswith('P_') and col.endswith('_last')]
    S_columns = [col for col in df.columns if col.startswith('S_') and col.endswith('_last')]
    B_columns = [col for col in df.columns if col.startswith('B_') and col.endswith('_last')]
    D_columns = [col for col in df.columns if col.startswith('D_') and col.endswith('_last')]
    R_columns = [col for col in df.columns if col.startswith('R_') and col.endswith('_last')]
    df['total_payments'] = df[P_columns].sum(axis=1)
    df['total_spending'] = df[S_columns].sum(axis=1)
    df['total_balance'] = df[B_columns].sum(axis=1)
    df['total_delinquency'] = df[D_columns].sum(axis=1)
    df['total_risk'] = df[R_columns].sum(axis=1)
    df['risk_to_payment_ratio'] = df['total_risk'] / (df['total_payments'] + 1e-6)
    df['delinquency_to_balance_ratio'] = df['total_delinquency'] / (df['total_balance'] + 1e-6)
    df=df.drop(columns=["total_delinquency", "total_payments"], errors='ignore')
    cols = list(df.dtypes[df.dtypes == 'float64'].index)
    for col in cols:
        if col=="customer_ID":
            continue;
        df[col] =df[col].astype(np.float32)
    cols = list(df.dtypes[df.dtypes == 'int64'].index)
    for col in cols:
        if col=="customer_ID":
            continue;
        df[col] =df[col].astype(np.int32)
    print('Shape after feature engineering:', df.shape)
    return df


In [12]:
trainBase=more_agg_inter_intra_variables(trainBase)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result 

Shape after feature engineering: (458913, 2818)


In [13]:
missing_columns_derived=missing_columns(trainBase)

In [14]:
trainBase=trainBase.drop(columns=missing_columns_derived, errors='ignore')

In [15]:
trainBase.shape

(458913, 2818)

In [16]:
from scipy.stats import pointbiserialr

def find_low_point_biserial_correlations_columns(df, target_column, category, suffix):

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(-127, inplace=True)


    columns_to_see = [col for col in df.columns if (col.startswith(category) and col.endswith(suffix)) or col == target_column]
    df = df[columns_to_see]

    correlation_results = []


    for col in columns_to_see:
        if col != target_column:
            correlation, p_value = pointbiserialr(df[target_column], df[col])
            correlation_results.append((col, correlation, p_value))


    correlation_df = pd.DataFrame(correlation_results, columns=['Feature', 'Point-Biserial Correlation', 'p-value'])


    filtered_df = correlation_df[(correlation_df['Point-Biserial Correlation'].abs() < point_biserial_threshold) & (correlation_df['p-value'] < stat_sig)]


    return filtered_df['Feature'].tolist()

In [17]:
category= ["B","P","S","D","R"]
suffix=['first', 'mean', 'std', 'min', 'max', 'last', 'first_last_lag_sub', 'first_last_lag_div', 'mean_last_lag_sub', 'mean_last_lag_div', 'diff1', 'last_round2', 'first_last_lag_sub_round2', 'first_last_lag_div_round2', 'mean_last_lag_sub_round2', 'mean_last_lag_div_round2',"log_last"]

In [18]:
def remove_low_correlation_columns(df):
    all_cols_to_omit=[]
    for c in category:
        for s in suffix:
            columns_to_omit=find_low_point_biserial_correlations_columns(df,"target",c,s)
            all_cols_to_omit.append(columns_to_omit)
    return all_cols_to_omit

In [19]:
all_low_correlation_columns=remove_low_correlation_columns(trainBase)

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5535: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rpb, prob = pearsonr(x, y)


In [20]:
for l in all_low_correlation_columns:
  trainBase.drop(columns=l, errors='ignore',inplace=True)

In [21]:
trainBase.shape

(458913, 2564)

In [22]:
from scipy.stats import pearsonr

def find_highly_correlated_columns(df, category, suffix):
    # df=df.to_pandas()
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(-127, inplace=True)


    numerical_columns = [col for col in df.columns if col.startswith(category) and col.endswith(suffix)]
    df = df[numerical_columns]


    correlation_matrix = df.corr(method='pearson').abs()


    correlated_pairs = set()
    columns_to_remove = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            if correlation_matrix.iloc[i, j] > correlation_threshold:
                col1 = correlation_matrix.columns[i]
                col2 = correlation_matrix.columns[j]


                correlated_pairs.add((col1, col2))

                columns_to_remove.add(col2)

    return list(columns_to_remove)

In [23]:
def remove_high_correlation_columns(df):
    all_cols_to_omit=[]
    for c in category:
        for s in suffix:
            columns_to_omit=find_highly_correlated_columns(df,c,s)
            all_cols_to_omit.append(columns_to_omit)
    return all_cols_to_omit

In [24]:
all_high_correlation_columns=remove_high_correlation_columns(trainBase)

In [25]:
for l in all_high_correlation_columns:
  trainBase.drop(columns=l, errors='ignore',inplace=True)

In [26]:
trainBase.shape

(458913, 1899)

In [28]:
trainBase.to_parquet("train_fe_v4_preprocessed_further_woLast.parquet")

In [29]:
del trainBase
_=gc.collect()

In [30]:
testBase = load_data(TEST)

Shape on loading:  (924621, 2709)


In [31]:
testBase=drop_columns_with_many_missing_values_in_derived_dataset(testBase,missing_cols_original)

In [32]:
testBase=more_agg_inter_intra_variables(testBase)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result 

Shape after feature engineering: (924621, 2817)


In [33]:
testBase.shape

(924621, 2817)

In [34]:
testBase=testBase.drop(columns=missing_columns_derived, errors='ignore')

In [35]:
testBase.shape

(924621, 2817)

In [36]:
for l in all_low_correlation_columns:
  testBase.drop(columns=l, errors='ignore',inplace=True)

In [37]:
testBase.shape

(924621, 2563)

In [38]:
for l in all_high_correlation_columns:
  testBase.drop(columns=l, errors='ignore',inplace=True)

In [39]:
testBase.shape

(924621, 1898)

In [40]:
testBase.to_parquet("test_fe_v4_preprocessed_further_woLast.parquet")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
del testBase
_=gc.collect()

In [ ]:
## DID NOT USE DUE TO OOM- WAS SUPPOSED TO BE USED ON NATIVE COMPETITION FILES AND THEN COMBINED WITH REMAINING FEATURES ALREADY CREATED IN AMEX-BASE
def more_agg_inter_intra_variables(df):

    df = df.fillna(-127)
    for col in df.select_dtypes(include='float64').columns:
        df[col] = df[col].astype(np.float32)
    for col in df.select_dtypes(include='int64').columns:
        df[col] = df[col].astype(np.int32)

    num_features = [col for col in df.columns if col not in ['customer_ID', 'S_2', 'target']]
    cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
    num_features = [col for col in num_features if col not in cat_features]
    final_df_list = []
    chunk_size = 100000
    unique_customers = df['customer_ID'].unique()

    for start in range(0, len(unique_customers), chunk_size):
        chunk_customer_ids = unique_customers[start:start + chunk_size]
        chunk_df = df[df['customer_ID'].isin(chunk_customer_ids)]

        agg_df = chunk_df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last', 'sum'])
        agg_df.columns = ['_'.join(x) for x in agg_df.columns]

        for col in num_features:
            agg_df[f'{col}_range'] = agg_df[f'{col}_max'] - agg_df[f'{col}_min']
            agg_df[f'{col}_cv'] = agg_df[f'{col}_std'] / (agg_df[f'{col}_mean'] + 1e-6)
            agg_df[f'{col}_iqr'] = chunk_df.groupby("customer_ID")[col].quantile(0.75) - chunk_df.groupby("customer_ID")[col].quantile(0.25)
            agg_df[f'{col}_log_mean'] = np.log1p(agg_df[f'{col}_mean'])
            agg_df[f'{col}_last_log'] = np.log1p(agg_df[f'{col}_last'])

        columns_to_drop = [f"{col}_{agg}" for col in num_features for agg in ['mean', 'std', 'min', 'max']]
        agg_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

        final_df_list.append(agg_df)

    final_df = pd.concat(final_df_list, axis=0)


    P_columns = [col for col in final_df.columns if col.startswith('P_') and col.endswith('_last')]
    S_columns = [col for col in final_df.columns if col.startswith('S_') and col.endswith('_last')]
    B_columns = [col for col in final_df.columns if col.startswith('B_') and col.endswith('_last')]
    D_columns = [col for col in final_df.columns if col.startswith('D_') and col.endswith('_last')]
    R_columns = [col for col in final_df.columns if col.startswith('R_') and col.endswith('_last')]

    final_df['total_payments'] = final_df[P_columns].sum(axis=1)
    final_df['total_spending'] = final_df[S_columns].sum(axis=1)
    final_df['total_balance'] = final_df[B_columns].sum(axis=1)
    final_df['total_delinquency'] = final_df[D_columns].sum(axis=1)
    final_df['total_risk'] = final_df[R_columns].sum(axis=1)

    final_df['risk_to_payment_ratio'] = final_df['total_risk'] / (final_df['total_payments'] + 1e-6)
    final_df['delinquency_to_balance_ratio'] = final_df['total_delinquency'] / (final_df['total_balance'] + 1e-6)

    final_df.drop(columns=["total_delinquency", "total_payments"], inplace=True, errors='ignore')

    print('Shape after feature engineering:', final_df.shape)

    return final_df